In [1]:
import numpy as np 
import re 
import os 
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import requests
import urllib.request,sys,time ,requests
import warnings
warnings.filterwarnings('ignore')
import contractions

In [2]:
from nltk.tokenize import sent_tokenize

In [3]:
stopWordsFile =     'StopWords_Generic.txt'
positiveWordsFile = 'positive-words.txt'
nagitiveWordsFile = 'negative-words.txt'

In [4]:
data= pd.read_excel('Input.xlsx')
data

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...
110,147.0,https://insights.blackcoffer.com/the-future-of...
111,148.0,https://insights.blackcoffer.com/big-data-anal...
112,149.0,https://insights.blackcoffer.com/business-anal...


In [5]:
data['URL']

0      https://insights.blackcoffer.com/ai-in-healthc...
1      https://insights.blackcoffer.com/what-if-the-c...
2      https://insights.blackcoffer.com/what-jobs-wil...
3      https://insights.blackcoffer.com/will-machine-...
4      https://insights.blackcoffer.com/will-ai-repla...
                             ...                        
109    https://insights.blackcoffer.com/blockchain-fo...
110    https://insights.blackcoffer.com/the-future-of...
111    https://insights.blackcoffer.com/big-data-anal...
112    https://insights.blackcoffer.com/business-anal...
113    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 114, dtype: object

In [6]:
corps=[]
TITLE=[]
LINK=[]


for url in data['URL']:
    
    page=requests.get(url,headers={"User-Agent": "XY"})
    if(page.status_code==200):
        #print('data fetched successfully',url)
        soup=BeautifulSoup(page.content,'html.parser')
        title=soup.find("h1",attrs = {'class':'entry-title'}).get_text()
        #print('title',title)
        text=soup.find(attrs ={'class':'td-post-content'}).get_text()
        lines=(line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text ='\n'.join(chunk for chunk in chunks if chunk)
        
        corps.append(text)
        TITLE.append(title)
        LINK.append(url)
       
    else:
        print('url not found',url)
        
        

url not found https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
url not found https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
url not found https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/


In [7]:
df = pd.DataFrame({'URL':LINK,'title':TITLE,'corps': corps})

In [8]:
df

,URL,title,corps
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ..."
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...
...,...,...,...
106,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of an...
107,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment?\nAn investment is a res...
108,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...
109,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...


In [9]:
def transform(text):

    review = re.sub('[^a-zA-Z0-9]', ' ',text)  # except small and capital letters and numeric remove everythong.
    review = review.lower()                    # lower it.
    review = review.split()
    
    review = [word for word in review if not word in stopwords.words('english')]   # remove stopwords.
    review = ' '.join(review)
    return review


df['Transform_Text'] = df['corps'].apply(transform)

In [10]:
# word count in each text row.

df['word_counts'] = df['Transform_Text'].apply(lambda x: len(str(x).split()))  

In [11]:
df['average number of words per sentence'] = np.nan

for i in range(0,len(df)):
    
    df['average number of words per sentence'][i] = df['word_counts'][i]/len(nltk.sent_tokenize(df['corps'][i]))

In [12]:
#Average Word Length is calculated by the formula:

#( Sum of the total number of characters in each word ) / ( Total number of words )

In [13]:
def char_count(x):
    s = x.split()
    x = ''.join(s)
    return len(x)      # counting the total number of characters in each text data.

In [14]:
df['chara_count'] = df['Transform_Text'].apply(lambda x: char_count(x))

In [15]:
df['average word length'] = np.nan

for i in range(0,len(df)):
    
    df['average word length'][i] = df['chara_count'][i]/df['word_counts'][i]

In [16]:
#SYLLABLE COUNT

In [17]:
def syllable_count(x):
    v = []
    d = {}
    for i in x:
        if i in "aeiou":
            v.append(i)
            d[i] = d.get(i,0)+1
            
    k = []
    for i in d:
        k.append(d[i])

    return np.sum(k)



In [18]:
df['syllable count'] = df['Transform_Text'].apply(lambda x: syllable_count(x))

In [19]:
#COMPLEX Word Count

In [20]:
from collections import  Counter

def complex_word_count(x):
    
    syllable = 'aeiou'
    
    t = x.split()
    
    v = []
    
    for i in t:
        words = i.split()
        c=Counter()
        
        for word in words:
            c.update(set(word))

        n = 0
        for a in c.most_common():
            if a[0] in syllable:
                if a[1] >= 2:
                    n += 1
                
        m = 0
        p = []
        for a in c.most_common():
            if a[0] in syllable:
                p.append(a[0])
        if len(p) >= 2:
            m += 1
        
        if n >= 1 or m >= 1:
            v.append(i)
            
    return len(v) 


In [21]:
df['complex_count'] = np.nan

df['complex_count'] = df['Transform_Text'].apply(lambda x: complex_word_count(x))
df.head()

,URL,title,corps,Transform_Text,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,introduction anything kills 10 million people ...,1171,15.207792,8253,7.047822,3174,940
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",human minds fascination carrying potential tin...,763,9.537500,4888,6.406291,1892,549
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,introduction ai rapidly evolving employment se...,1005,11.823529,7244,7.207960,2828,825
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,anything could give rise smarter human intelli...,914,9.621053,5786,6.330416,2263,689
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,machine intelligence last invention humanity e...,1010,12.784810,6696,6.629703,2502,735


In [22]:
df['sentence length'] = np.nan
df['Average Sentence Length'] = np.nan
df['Percentage of Complex words'] = np.nan
df['Fog Index'] = np.nan

In [23]:
for i in range(0,len(df)):
    
    df['sentence length'][i]  =   len(nltk.sent_tokenize(df['corps'][i]))
    df['Average Sentence Length'][i] = df['word_counts'][i]/df['sentence length'][i]
    df['Percentage of Complex words'][i] = df['complex_count'][i]/df['word_counts'][i] 
    df['Fog Index'][i] = 0.4 * (df['Average Sentence Length'][i] + df['Percentage of Complex words'][i])

In [24]:
#subjective score


In [25]:

from textblob import TextBlob

In [26]:
df['subjectivity'] = np.nan

for i in range(len(df)):
    df['subjectivity'][i] = TextBlob(df['corps'][i]).sentiment[1]

In [27]:
df.head()

,URL,title,corps,Transform_Text,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,subjectivity
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,introduction anything kills 10 million people ...,1171,15.207792,8253,7.047822,3174,940,77.0,15.207792,0.802733,6.404210,0.463364
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",human minds fascination carrying potential tin...,763,9.537500,4888,6.406291,1892,549,80.0,9.537500,0.719528,4.102811,0.433596
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,introduction ai rapidly evolving employment se...,1005,11.823529,7244,7.207960,2828,825,85.0,11.823529,0.820896,5.057770,0.483201
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,anything could give rise smarter human intelli...,914,9.621053,5786,6.330416,2263,689,95.0,9.621053,0.753829,4.149953,0.487387
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,machine intelligence last invention humanity e...,1010,12.784810,6696,6.629703,2502,735,79.0,12.784810,0.727723,5.405013,0.509139


In [28]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [29]:
df['PERSONAL PRONOUNS'] = np.nan

for i in range(len(df)):
    doc = nlp(df['corps'][i])
    tok = []
    for token in doc:
        if token.pos_ == 'PRON':
            tok.append(token)
        
    df['PERSONAL PRONOUNS'][i] = tok

In [30]:
df

,URL,title,corps,Transform_Text,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,subjectivity,PERSONAL PRONOUNS
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,introduction anything kills 10 million people ...,1171,15.207792,8253,7.047822,3174,940,77.0,15.207792,0.802733,6.404210,0.463364,"[anything, it, us, it, our, our, it, that, It,..."
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",human minds fascination carrying potential tin...,763,9.537500,4888,6.406291,1892,549,80.0,9.537500,0.719528,4.102811,0.433596,"[itself, anything, it, It, something, it, this..."
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,introduction ai rapidly evolving employment se...,1005,11.823529,7244,7.207960,2828,825,85.0,11.823529,0.820896,5.057770,0.483201,"[their, that, This, which, that, which, them, ..."
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,anything could give rise smarter human intelli...,914,9.621053,5786,6.330416,2263,689,95.0,9.621053,0.753829,4.149953,0.487387,"[Anything, that, Nothing, There, our, they, Th..."
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,machine intelligence last invention humanity e...,1010,12.784810,6696,6.629703,2502,735,79.0,12.784810,0.727723,5.405013,0.509139,"[it, us, anything, it, they, which, It, that, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of an...,reconciling financial realities mba education ...,567,11.571429,3852,6.793651,1387,427,49.0,11.571429,0.753086,4.929806,0.402814,"[There, ’s, my, I, It, me, I, my, who, who, it..."
107,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment?\nAn investment is a res...,investment investment resource thing procured ...,1007,16.241935,6329,6.285005,2359,684,62.0,16.241935,0.679245,6.768472,0.391713,"[What, that, which, one, One, something, those..."
108,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...,quality affordable healthcare vision governmen...,690,10.454545,4632,6.713043,1779,517,66.0,10.454545,0.749275,4.481528,0.417892,"[it, Some, everyone, that, that, he, she, it, ..."
109,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...,analytics statistical scientific process disco...,430,14.827586,3240,7.534884,1234,344,29.0,14.827586,0.800000,6.251034,0.546448,"[that, that, this, itself, their, your, their,..."


In [31]:
#sentiment

In [32]:
#nltk.download('vader_lexicon')

In [33]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [34]:
# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()


In [35]:
df.head(10)

,URL,title,corps,Transform_Text,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,subjectivity,PERSONAL PRONOUNS
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,introduction anything kills 10 million people ...,1171,15.207792,8253,7.047822,3174,940,77.0,15.207792,0.802733,6.404210,0.463364,"[anything, it, us, it, our, our, it, that, It,..."
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",human minds fascination carrying potential tin...,763,9.537500,4888,6.406291,1892,549,80.0,9.537500,0.719528,4.102811,0.433596,"[itself, anything, it, It, something, it, this..."
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,introduction ai rapidly evolving employment se...,1005,11.823529,7244,7.207960,2828,825,85.0,11.823529,0.820896,5.057770,0.483201,"[their, that, This, which, that, which, them, ..."
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,anything could give rise smarter human intelli...,914,9.621053,5786,6.330416,2263,689,95.0,9.621053,0.753829,4.149953,0.487387,"[Anything, that, Nothing, There, our, they, Th..."
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,machine intelligence last invention humanity e...,1010,12.784810,6696,6.629703,2502,735,79.0,12.784810,0.727723,5.405013,0.509139,"[it, us, anything, it, they, which, It, that, ..."
5,https://insights.blackcoffer.com/man-and-machi...,Will machine replace the human in the future o...,Introduction\nWhere is this disruptive technol...,introduction disruptive technology taking us t...,702,10.968750,4719,6.722222,1779,517,64.0,10.968750,0.736467,4.682087,0.453334,"[us, it, it, You, that, them, I, I, it, what, ..."
6,https://insights.blackcoffer.com/in-future-or-...,How humans and machines are evolving to work t...,In future or in upcoming years humans and mach...,future upcoming years humans machines going wo...,421,9.355556,2778,6.598575,1051,310,45.0,9.355556,0.736342,4.036759,0.416908,"[which, he, there, that, They, our, their, we,..."
7,https://insights.blackcoffer.com/how-machine-l...,How machine learning will affect your business?,Machine learning techniques may have been used...,machine learning techniques may used years rec...,408,11.333333,2549,6.247549,937,285,36.0,11.333333,0.698529,4.812745,0.566437,"[there, their, which, we, we, everything, they..."
8,https://insights.blackcoffer.com/deep-learning...,Deep learning impact on areas of e-learning?,eLearning as technology becomes more affordabl...,elearning technology becomes affordable higher...,1221,15.074074,7913,6.480753,3113,931,81.0,15.074074,0.762490,6.334626,0.531156,"[its, It, which, that, its, that, It, you, you..."
9,https://insights.blackcoffer.com/how-to-protec...,How to protect future data and its privacy?,"Before the internet, information was in some w...",internet information ways restricted centraliz...,1177,11.317308,7751,6.585387,2907,880,104.0,11.317308,0.747664,4.825988,0.465092,"[it, YOUR, your, it, anyone, that, it, that, T..."


In [36]:
def positive_score(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)["pos"]

In [37]:
def negative_score(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)["neg"]


In [38]:
def polarity(text):
    return TextBlob(text).sentiment[0]

In [39]:
df["positive_score"] = df["corps"].apply(positive_score)

In [40]:
df["negative_score"] = df["corps"].apply(negative_score)

In [41]:
df["polarity_score"] = df["corps"].apply(polarity)

In [42]:
df

,URL,title,corps,Transform_Text,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,subjectivity,PERSONAL PRONOUNS,positive_score,negative_score,polarity_score
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction\n“If anything kills over 10 milli...,introduction anything kills 10 million people ...,1171,15.207792,8253,7.047822,3174,940,77.0,15.207792,0.802733,6.404210,0.463364,"[anything, it, us, it, our, our, it, that, It,...",0.145,0.044,0.136936
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying ...",human minds fascination carrying potential tin...,763,9.537500,4888,6.406291,1892,549,80.0,9.537500,0.719528,4.102811,0.433596,"[itself, anything, it, It, something, it, this...",0.149,0.084,0.074693
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction\nAI is rapidly evolving in the em...,introduction ai rapidly evolving employment se...,1005,11.823529,7244,7.207960,2828,825,85.0,11.823529,0.820896,5.057770,0.483201,"[their, that, This, which, that, which, them, ...",0.075,0.036,0.111470
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-than...,anything could give rise smarter human intelli...,914,9.621053,5786,6.330416,2263,689,95.0,9.621053,0.753829,4.149953,0.487387,"[Anything, that, Nothing, There, our, they, Th...",0.145,0.044,0.142050
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention th...,machine intelligence last invention humanity e...,1010,12.784810,6696,6.629703,2502,735,79.0,12.784810,0.727723,5.405013,0.509139,"[it, us, anything, it, they, which, It, that, ...",0.147,0.050,0.053178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,Reconciling with the financial realities of an...,reconciling financial realities mba education ...,567,11.571429,3852,6.793651,1387,427,49.0,11.571429,0.753086,4.929806,0.402814,"[There, ’s, my, I, It, me, I, my, who, who, it...",0.129,0.051,0.035488
107,https://insights.blackcoffer.com/the-future-of...,The future of Investing,What Is an Investment?\nAn investment is a res...,investment investment resource thing procured ...,1007,16.241935,6329,6.285005,2359,684,62.0,16.241935,0.679245,6.768472,0.391713,"[What, that, which, one, One, something, those...",0.110,0.020,0.076504
108,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,Quality and affordable healthcare is a vision ...,quality affordable healthcare vision governmen...,690,10.454545,4632,6.713043,1779,517,66.0,10.454545,0.749275,4.481528,0.417892,"[it, Some, everyone, that, that, he, she, it, ...",0.091,0.092,0.040319
109,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,Analytics is a statistical scientific process ...,analytics statistical scientific process disco...,430,14.827586,3240,7.534884,1234,344,29.0,14.827586,0.800000,6.251034,0.546448,"[that, that, this, itself, their, your, their,...",0.170,0.004,0.208748


In [46]:
df.drop(['corps', 'Transform_Text'],axis=1, inplace=True)

In [47]:
df

,URL,title,word_counts,average number of words per sentence,chara_count,average word length,syllable count,complex_count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,subjectivity,PERSONAL PRONOUNS,positive_score,negative_score,polarity_score
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,1171,15.207792,8253,7.047822,3174,940,77.0,15.207792,0.802733,6.404210,0.463364,"[anything, it, us, it, our, our, it, that, It,...",0.145,0.044,0.136936
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,763,9.537500,4888,6.406291,1892,549,80.0,9.537500,0.719528,4.102811,0.433596,"[itself, anything, it, It, something, it, this...",0.149,0.084,0.074693
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,1005,11.823529,7244,7.207960,2828,825,85.0,11.823529,0.820896,5.057770,0.483201,"[their, that, This, which, that, which, them, ...",0.075,0.036,0.111470
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,914,9.621053,5786,6.330416,2263,689,95.0,9.621053,0.753829,4.149953,0.487387,"[Anything, that, Nothing, There, our, they, Th...",0.145,0.044,0.142050
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,1010,12.784810,6696,6.629703,2502,735,79.0,12.784810,0.727723,5.405013,0.509139,"[it, us, anything, it, they, which, It, that, ...",0.147,0.050,0.053178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments,567,11.571429,3852,6.793651,1387,427,49.0,11.571429,0.753086,4.929806,0.402814,"[There, ’s, my, I, It, me, I, my, who, who, it...",0.129,0.051,0.035488
107,https://insights.blackcoffer.com/the-future-of...,The future of Investing,1007,16.241935,6329,6.285005,2359,684,62.0,16.241935,0.679245,6.768472,0.391713,"[What, that, which, one, One, something, those...",0.110,0.020,0.076504
108,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare,690,10.454545,4632,6.713043,1779,517,66.0,10.454545,0.749275,4.481528,0.417892,"[it, Some, everyone, that, that, he, she, it, ...",0.091,0.092,0.040319
109,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry,430,14.827586,3240,7.534884,1234,344,29.0,14.827586,0.800000,6.251034,0.546448,"[that, that, this, itself, their, your, their,...",0.170,0.004,0.208748


In [48]:
df.to_csv('OUTPUT.csv')